# Ejemplo de uso de Selenium

Asignatura: Procesamiento del Lenguaje Natural

Fecha: 20/3/2025

Docentes:
- Ferrucci, Constantino
- Geary, Alan
- Manson, Juan Pablo
- Sollberger, Dolores

## Introducción

### Instalación de dependencias
En esta celda, actualizamos las dependencias del sistema y luego instalamos las librerías necesarias para trabajar con Selenium:  
- **Selenium**: para la automatización de navegadores web.  
- **webdriver-manager**: para gestionar automáticamente los controladores del navegador (Chrome, en este caso).

### Configuración del entorno y opciones para Selenium
En esta celda:
1. Importamos las librerías necesarias de Selenium y otras utilidades.
2. Configuramos opciones del navegador Chrome:
   - **--headless**: ejecuta Chrome sin interfaz gráfica.
   - **--no-sandbox**: evita restricciones de seguridad en entornos virtualizados.
   - **--disable-dev-shm-usage**: mejora la gestión de memoria compartida en Docker.
3. Especificamos la ruta al controlador de Chrome.

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Configuración más detallada de las opciones de Chrome
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')

# Configurar la ruta del driver
service = Service('/usr/bin/chromedriver')

### Función para extraer videos de YouTube
Esta función permite:
1. Esperar un tiempo opcional para cargar el contenido de la página.
2. Extraer información sobre los primeros 5 videos:
   - **Título**.
   - **Nombre del canal**.
   - **Likes** y **tiempo de publicación**.
3. Imprimir los resultados de forma ordenada.

In [ ]:
def extraer_videos_youtube(driver, esperar_carga=True, delay=5, num_of_videos=10):
    """
    Extrae y muestra los títulos, canales y metadata de los videos en una página de resultados de YouTube.

    Args:
        driver (webdriver.Chrome): Instancia del navegador.
        esperar_carga (bool): Si True, espera un tiempo definido para que cargue la página.
        delay (int): Tiempo en segundos a esperar si esperar_carga es True.

    Returns:
        None
    """
    # Verificar si se debe esperar por la carga de la página
    if esperar_carga:
        print(f"Esperando {delay} segundos para que cargue la página...")
        time.sleep(delay)

    # Intentar extraer los videos
    videos = driver.find_elements(By.TAG_NAME, "ytd-video-renderer")

    # Verificar si se encontraron videos
    if not videos:
        print("No se encontraron videos en la página.")
        return

    # Imprimir encabezado
    print(f"Títulos de los primeros {min(num_of_videos, len(videos))} videos encontrados:")
    print("-" * 50)

    # Iterar y mostrar detalles de los videos
    for i, video in enumerate(videos[:num_of_videos], start=1):
        # Extraer detalles del video
        titulo = video.find_element(By.ID, "video-title").text
        canal = video.find_element(By.ID, "channel-name").find_element(By.CSS_SELECTOR, 'ytd-channel-name a').get_attribute('innerText')
        metadata = video.find_element(By.ID, "metadata-line").text.split('\n')
        likes = metadata[0]
        tiempo = metadata[1]

        # Imprimir los detalles de cada video
        print(f"Video {i}:")
        print(f"  Título: {titulo}")
        print(f"  Canal: {canal}")
        print(f"  Likes: {likes}")
        print(f"  Tiempo: {tiempo}")
        print("-" * 50)

### Inicialización del navegador y ejecución
1. **Navegador Chrome**:
   - Se inicializa con las opciones configuradas previamente.
2. **Navegar a YouTube**:
   - URL de búsqueda: `openai gpt 3.5 turbo`.
3. **Ejecutar la función `extraer_videos_youtube`**:
   - Se llama a la función para extraer los detalles de los videos.
4. **Liberar recursos**:
   - Cierra el navegador con `driver.quit()`.

Prueba sin tiempo de espera

In [ ]:
# Inicializar el webdriver
driver = webdriver.Chrome(options=chrome_options)

# Navegar a la página de YouTube
driver.get('https://www.youtube.com/results?search_query=openai+gpt+3.5+turbo')

# Llamar a la función para extraer videos
extraer_videos_youtube(driver, esperar_carga=False, num_of_videos=15)

# Cerrar el navegador
driver.quit()

No se encontraron videos en la página.


Prueba con tiempo de espera

In [ ]:
# Inicializar el webdriver
driver = webdriver.Chrome(options=chrome_options)

# Navegar a la página de YouTube
driver.get('https://www.youtube.com/results?search_query=openai+gpt+3.5+turbo')

# Llamar a la función para extraer videos
extraer_videos_youtube(driver, esperar_carga=True, delay=5, num_of_videos=5)

# Cerrar el navegador
driver.quit()

Esperando 5 segundos para que cargue la página...
Títulos de los primeros 5 videos encontrados:
--------------------------------------------------
Video 1:
  Título: OpenAI GPT 3.5 Turbo Explained in 100 Seconds
  Canal: Ai Austin
  Likes: 7.5K views
  Tiempo: 2 years ago
--------------------------------------------------
Video 2:
  Título: Getting started with the NEW OpenAI ChatGPT API (gpt-3.5-turbo)
  Canal: Tinkering with Deep Learning & AI
  Likes: 34K views
  Tiempo: 2 years ago
--------------------------------------------------
Video 3:
  Título: How to Fine Tune GPT 3.5 Turbo with OpenAI's API
  Canal: CreateContentWithGenAI
  Likes: 296 views
  Tiempo: 1 year ago
--------------------------------------------------
Video 4:
  Título: How to Use OpenAI API to Communicate with GPT-3.5 Turbo Model
  Canal: Tom Media
  Likes: 562 views
  Tiempo: 1 year ago
--------------------------------------------------
Video 5:
  Título: OpenAI Fights Back (GPT 4.5 is wild)
  Canal: Theo - t3․g

### Ejemplo con requests

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

def extraer_videos_youtube(url, num_of_videos=10):
    """
    Extrae y muestra los títulos, canales y metadata de los videos en una página de resultados de YouTube
    utilizando requests y BeautifulSoup.

    Args:
        url (str): URL de la página de resultados de YouTube.
        num_of_videos (int): Número máximo de videos a extraer.

    Returns:
        list: Lista de diccionarios con información de cada video.
    """
    # Configurar headers para simular un navegador
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept-Language': 'es-ES,es;q=0.9'
    }

    # Realizar la solicitud HTTP
    response = requests.get(url, headers=headers)

    # Verificar si la solicitud fue exitosa
    if response.status_code != 200:
        print(f"Error al acceder a la página. Código de estado: {response.status_code}")
        return []

    # Parsear el HTML con BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Lista para almacenar la información de los videos
    videos_info = []

    # Intentamos encontrar los contenedores principales de videos
    videos = soup.select('div#contents ytd-video-renderer')

    # Si no encontramos videos con el selector anterior, intentamos otro enfoque
    if not videos:
        # Alternativa: buscar secciones que contengan videos
        videos = soup.find_all('div', {'id': re.compile('dismissible')})

    # Verificar si se encontraron videos
    if not videos:
        print("No se encontraron videos en la página.")
        return []

    # Imprimir encabezado
    print(f"Títulos de los primeros {min(num_of_videos, len(videos))} videos encontrados:")
    print("-" * 50)

    # Limitar la cantidad de videos a procesar
    videos = videos[:num_of_videos]

    # Iterar y extraer detalles de los videos
    for i, video in enumerate(videos, start=1):
        try:
            # Intentamos diferentes selectores para adaptarnos a la estructura de YouTube
            # Título del video
            titulo_elem = video.select_one('a#video-title') or video.select_one('h3 a')
            titulo = titulo_elem.text.strip() if titulo_elem else "Título no disponible"

            # Canal
            canal_elem = video.select_one('div#channel-info a') or video.select_one('ytd-channel-name a')
            canal = canal_elem.text.strip() if canal_elem else "Canal no disponible"

            # Metadatos (vistas, tiempo)
            metadata_elem = video.select_one('div#metadata-line') or video.select_one('span.style-scope.ytd-video-meta-block')

            if metadata_elem:
                metadata_text = metadata_elem.text.strip()
                # Intentar extraer vistas y tiempo
                metadata_parts = metadata_text.split('\n')
                vistas = metadata_parts[0].strip() if len(metadata_parts) > 0 else "Vistas no disponibles"
                tiempo = metadata_parts[1].strip() if len(metadata_parts) > 1 else "Tiempo no disponible"
            else:
                vistas = "Vistas no disponibles"
                tiempo = "Tiempo no disponible"

            # Crear diccionario con la información del video
            video_info = {
                'titulo': titulo,
                'canal': canal,
                'vistas': vistas,
                'tiempo': tiempo
            }

            # Agregar a la lista de videos
            videos_info.append(video_info)

            # Imprimir los detalles del video
            print(f"Video {i}:")
            print(f"  Título: {titulo}")
            print(f"  Canal: {canal}")
            print(f"  Vistas: {vistas}")
            print(f"  Tiempo: {tiempo}")
            print("-" * 50)

        except Exception as e:
            print(f"Error al procesar el video {i}: {str(e)}")

    return videos_info

In [ ]:
url = "https://www.youtube.com/results?search_query=python+tutorial"
extraer_videos_youtube(url, num_of_videos=5)

No se encontraron videos en la página.


[]

## Ejemplo

### Levantamos un servidor con un HTML

In [ ]:
from flask import Flask, render_template_string, jsonify
import threading
import requests
import time
import logging

app = Flask(__name__)

# Función para obtener el valor del dólar
def obtener_dolar():
    try:
        response = requests.get("https://dolarapi.com/v1/dolares")
        if response.status_code == 200:
            return response.json()
        else:
            return {"error": "No se pudo obtener el valor del dólar"}
    except Exception as e:
        return {"error": str(e)}

# Página HTML dinámica
html_code = """
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Dólar API</title>
    <link rel="preconnect" href="https://fonts.googleapis.com">
    <link href="https://fonts.googleapis.com/css2?family=Poppins:wght@300;400;600&display=swap" rel="stylesheet">
    <style>
        /* Estilos generales */
        body {
            font-family: 'Poppins', sans-serif;
            background-color: #f4f4f9;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            margin: 0;
        }

        /* Contenedor principal */
        .container {
            background: #ffffff;
            padding: 20px 30px;
            border-radius: 12px;
            box-shadow: 0 5px 15px rgba(0, 0, 0, 0.1);
            text-align: center;
            max-width: 400px;
            width: 100%;
        }

        h1 {
            font-size: 22px;
            color: #333;
            margin-bottom: 10px;
        }

        /* Lista de valores */
        .dolar-list {
            list-style: none;
            padding: 0;
        }

        .dolar-list li {
            background: #007bff;
            color: white;
            padding: 12px;
            margin: 8px 0;
            border-radius: 8px;
            font-size: 16px;
            font-weight: 600;
        }

        /* Animación de carga */
        .loading {
            font-size: 16px;
            color: #777;
            margin-top: 10px;
        }

        .loading::after {
            content: "⏳";
            animation: dots 1.5s infinite;
        }

        @keyframes dots {
            0% { content: "⏳"; }
            33% { content: "⌛"; }
            66% { content: "⏳"; }
            100% { content: "⌛"; }
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>Valores del Dólar 💰</h1>
        <ul class="dolar-list" id="lista-dolar">
            <li class="loading">Cargando datos...</li>
        </ul>
    </div>

    <script>
        // Esperar 3 segundos antes de hacer la petición
        setTimeout(() => {
            fetch('https://dolarapi.com/v1/dolares')
                .then(response => response.json())
                .then(data => {
                    const lista = document.getElementById("lista-dolar");
                    lista.innerHTML = ""; // Limpiar la lista

                    // Agregar cada tipo de dólar a la lista
                    data.forEach(d => {
                        const li = document.createElement("li");
                        li.textContent = `${d.nombre}: Compra ${d.compra} | Venta ${d.venta}`;
                        lista.appendChild(li);
                    });
                })
                .catch(error => {
                    console.error("Error al obtener los datos:", error);
                    document.getElementById("lista-dolar").innerHTML = "<li>Error al cargar los datos</li>";
                });
        }, 3000);
    </script>
</body>
</html>
"""

@app.route('/')
def home():
    return render_template_string(html_code)

@app.route('/api/dolar')
def dolar():
    return jsonify(obtener_dolar())

# Iniciar Flask en un hilo separado
def run_flask():
    # Configurar el nivel de logging de Flask
    log = logging.getLogger('werkzeug')
    log.setLevel(logging.ERROR)  # Oculta logs de peticiones normales, pero muestra errores
    app.run(host='0.0.0.0', port=5000)

flask_thread = threading.Thread(target=run_flask)
flask_thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


### Definimos las funciones

#### Requests

In [ ]:
import requests
from bs4 import BeautifulSoup

def obtener_dolar_requests():
    """
    Obtiene el valor del dólar desde una página HTML estática con Requests y BeautifulSoup.
    """
    url = "http://localhost:5000"
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        # Buscar los valores del dólar en la lista
        dolares = soup.find_all("li")

        if dolares:
            print("💰 Valores del dólar (Requests - Estático):")
            for dolar in dolares:
                print(dolar.text)
        else:
            print("⚠ No se encontraron valores del dólar en la página.")

    else:
        print(f"❌ Error al conectar con la página. Código: {response.status_code}")

#### Selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

def obtener_dolar_solo_selenium(delay=5):
    """
    Obtiene el valor del dólar desde una página dinámica usando solo Selenium.
    """
    # Configurar Selenium con WebDriver Manager
    driver = webdriver.Chrome(options=chrome_options)

    try:
        url = "http://localhost:5000"
        driver.get(url)

        print(f"⏳ Esperando {delay} segundos para que carguen los datos...")
        time.sleep(delay)  # Esperar para permitir que el JavaScript cargue los datos dinámicos

        # Buscar los elementos de la lista que contienen los valores del dólar
        dolares = driver.find_elements(By.TAG_NAME, "li")

        if dolares:
            print("💰 Valores del dólar (Selenium - Solo Selenium):")
            for dolar in dolares:
                print(dolar.text)
        else:
            print("⚠ No se encontraron valores del dólar en la página.")

    finally:
        driver.quit()  # Cerrar el navegador

### Probamos las funciones

#### De forma estática

In [ ]:
obtener_dolar_requests()

💰 Valores del dólar (Requests - Estático):
Cargando datos...


In [ ]:
obtener_dolar_solo_selenium(0)

⏳ Esperando 0 segundos para que carguen los datos...
💰 Valores del dólar (Selenium - Solo Selenium):
Cargando datos...


#### De forma dinámica

In [ ]:
obtener_dolar_solo_selenium(5)

⏳ Esperando 5 segundos para que carguen los datos...
💰 Valores del dólar (Selenium - Solo Selenium):
Oficial: Compra 1049.5 | Venta 1089.5
Blue: Compra 1245 | Venta 1265
Bolsa: Compra 1282.2 | Venta 1286.7
Contado con liquidación: Compra 1285.3 | Venta 1286.4
Mayorista: Compra 1066.5 | Venta 1069.5
Cripto: Compra 1289.9 | Venta 1295.1
Tarjeta: Compra 1364.35 | Venta 1416.35


#### Bajamos el servidor

In [ ]:
!pkill -9 flask

## Conclusión final

### Comparación entre Búsqueda Estática y Dinámica en Web Scraping

| Característica        | Búsqueda Estática | Búsqueda Dinámica |
|----------------------|------------------|------------------|
| **Método**          | `requests` + `BeautifulSoup` | `Selenium`, `Playwright` |
| **Velocidad**       | Rápida | Lenta |
| **Carga JavaScript** | No | Sí |
| **Consumo de recursos** | Bajo | Alto |
| **Dificultad**      | Fácil | Más compleja |
| **Riesgo de bloqueo** | Bajo | Alto |
| **Uso recomendado** | Datos visibles en el HTML | Datos cargados con JavaScript |


La búsqueda dinámica en web scraping es útil cuando los datos no están en el HTML y se generan con JavaScript. Aunque es más lenta y consume más recursos, permite extraer información que no se puede obtener con un método estático. Se debe usar solo cuando sea necesario y con cuidado para evitar bloqueos o problemas legales.
